# Minimal GPT from Scratch

**Learning Objectives:**
1. Build a decoder-only (autoregressive) transformer
2. Implement causal masking for left-to-right generation
3. Train on character-level Shakespeare text
4. Generate text samples during training
5. Track loss and perplexity curves

**Prerequisites:** [gpt](../transformers/gpt.md), [08-minimal-transformer](08-minimal-transformer.ipynb)

**Key Insight:** GPT is just a transformer decoder with causal masking. The model predicts the next token given all previous tokens.

**Reference:** Inspired by Karpathy's nanoGPT

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import urllib.request
import os

torch.manual_seed(42)
np.random.seed(42)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

## 1. Load Shakespeare Dataset

We'll use a small Shakespeare corpus for character-level language modeling.

In [ ]:
# Download Shakespeare text
data_url = 'https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt'
data_path = 'shakespeare.txt'

if not os.path.exists(data_path):
    print("Downloading Shakespeare text...")
    urllib.request.urlretrieve(data_url, data_path)

with open(data_path, 'r') as f:
    text = f.read()

print(f"Total characters: {len(text):,}")
print(f"\nFirst 500 characters:")
print(text[:500])

In [ ]:
# Create character-level vocabulary
chars = sorted(list(set(text)))
vocab_size = len(chars)

print(f"Vocabulary size: {vocab_size}")
print(f"Characters: {''.join(chars)}")

# Character to index mapping
char_to_idx = {ch: i for i, ch in enumerate(chars)}
idx_to_char = {i: ch for i, ch in enumerate(chars)}

# Encode/decode functions
def encode(s):
    return [char_to_idx[c] for c in s]

def decode(indices):
    return ''.join([idx_to_char[i] for i in indices])

# Test encoding
test_str = "Hello"
encoded = encode(test_str)
decoded = decode(encoded)
print(f"\n'{test_str}' -> {encoded} -> '{decoded}'")

In [ ]:
# Encode entire dataset
data = torch.tensor(encode(text), dtype=torch.long)
print(f"Data shape: {data.shape}")

# Train/val split
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

print(f"Train: {len(train_data):,} tokens")
print(f"Val: {len(val_data):,} tokens")

## 2. Dataset and Batching

In [ ]:
class TextDataset(Dataset):
    """Character-level text dataset."""
    
    def __init__(self, data, block_size):
        self.data = data
        self.block_size = block_size
    
    def __len__(self):
        return len(self.data) - self.block_size
    
    def __getitem__(self, idx):
        # Get a chunk of text
        x = self.data[idx:idx + self.block_size]
        y = self.data[idx + 1:idx + self.block_size + 1]
        return x, y

# Hyperparameters
block_size = 64  # Context length
batch_size = 64

train_dataset = TextDataset(train_data, block_size)
val_dataset = TextDataset(val_data, block_size)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

# Show example batch
x, y = train_dataset[0]
print(f"Input shape: {x.shape}")
print(f"Target shape: {y.shape}")
print(f"\nInput: {decode(x.tolist())}")
print(f"\nTarget (shifted by 1): {decode(y.tolist())}")

## 3. GPT Model Architecture

GPT is a **decoder-only** transformer:
- Uses **causal masking** (each position can only attend to earlier positions)
- **No encoder** - just stacked decoder blocks
- Predicts next token given all previous tokens

In [ ]:
class CausalSelfAttention(nn.Module):
    """Causal (masked) self-attention for autoregressive modeling."""
    
    def __init__(self, d_model, n_heads, block_size, dropout=0.1):
        super().__init__()
        assert d_model % n_heads == 0
        
        self.n_heads = n_heads
        self.d_k = d_model // n_heads
        self.d_model = d_model
        
        # Combined QKV projection for efficiency
        self.qkv = nn.Linear(d_model, 3 * d_model)
        self.proj = nn.Linear(d_model, d_model)
        
        self.attn_dropout = nn.Dropout(dropout)
        self.proj_dropout = nn.Dropout(dropout)
        
        # Causal mask: lower triangular
        # Position i can only attend to positions <= i
        mask = torch.tril(torch.ones(block_size, block_size))
        self.register_buffer('mask', mask.view(1, 1, block_size, block_size))
        
        self.scale = 1.0 / np.sqrt(self.d_k)
    
    def forward(self, x):
        B, T, C = x.shape  # batch, seq_len, d_model
        
        # Compute Q, K, V
        qkv = self.qkv(x)  # (B, T, 3*d_model)
        q, k, v = qkv.split(self.d_model, dim=2)
        
        # Reshape for multi-head attention
        q = q.view(B, T, self.n_heads, self.d_k).transpose(1, 2)  # (B, n_heads, T, d_k)
        k = k.view(B, T, self.n_heads, self.d_k).transpose(1, 2)
        v = v.view(B, T, self.n_heads, self.d_k).transpose(1, 2)
        
        # Attention scores
        attn = (q @ k.transpose(-2, -1)) * self.scale  # (B, n_heads, T, T)
        
        # Apply causal mask
        attn = attn.masked_fill(self.mask[:, :, :T, :T] == 0, float('-inf'))
        attn = F.softmax(attn, dim=-1)
        attn = self.attn_dropout(attn)
        
        # Weighted sum of values
        out = attn @ v  # (B, n_heads, T, d_k)
        out = out.transpose(1, 2).contiguous().view(B, T, C)  # (B, T, d_model)
        
        return self.proj_dropout(self.proj(out))

In [ ]:
class FeedForward(nn.Module):
    """Position-wise feedforward network."""
    
    def __init__(self, d_model, dropout=0.1):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(d_model, 4 * d_model),
            nn.GELU(),
            nn.Linear(4 * d_model, d_model),
            nn.Dropout(dropout),
        )
    
    def forward(self, x):
        return self.net(x)


class TransformerBlock(nn.Module):
    """GPT-style transformer block (pre-norm)."""
    
    def __init__(self, d_model, n_heads, block_size, dropout=0.1):
        super().__init__()
        self.ln1 = nn.LayerNorm(d_model)
        self.attn = CausalSelfAttention(d_model, n_heads, block_size, dropout)
        self.ln2 = nn.LayerNorm(d_model)
        self.ffn = FeedForward(d_model, dropout)
    
    def forward(self, x):
        x = x + self.attn(self.ln1(x))
        x = x + self.ffn(self.ln2(x))
        return x

In [ ]:
class GPT(nn.Module):
    """GPT Language Model."""
    
    def __init__(self, vocab_size, d_model, n_heads, n_layers, block_size, dropout=0.1):
        super().__init__()
        self.block_size = block_size
        
        # Token and position embeddings
        self.tok_emb = nn.Embedding(vocab_size, d_model)
        self.pos_emb = nn.Embedding(block_size, d_model)
        self.drop = nn.Dropout(dropout)
        
        # Transformer blocks
        self.blocks = nn.ModuleList([
            TransformerBlock(d_model, n_heads, block_size, dropout)
            for _ in range(n_layers)
        ])
        
        # Output
        self.ln_f = nn.LayerNorm(d_model)
        self.head = nn.Linear(d_model, vocab_size, bias=False)
        
        # Weight tying: share weights between embedding and output
        self.tok_emb.weight = self.head.weight
        
        # Initialize weights
        self.apply(self._init_weights)
    
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            nn.init.normal_(module.weight, mean=0.0, std=0.02)
    
    def forward(self, idx, targets=None):
        """
        Args:
            idx: (B, T) token indices
            targets: (B, T) target token indices (optional)
        Returns:
            logits: (B, T, vocab_size)
            loss: scalar if targets provided
        """
        B, T = idx.shape
        assert T <= self.block_size, f"Sequence length {T} exceeds block size {self.block_size}"
        
        # Get embeddings
        pos = torch.arange(0, T, device=idx.device).unsqueeze(0)  # (1, T)
        tok_emb = self.tok_emb(idx)  # (B, T, d_model)
        pos_emb = self.pos_emb(pos)  # (1, T, d_model)
        x = self.drop(tok_emb + pos_emb)
        
        # Pass through transformer blocks
        for block in self.blocks:
            x = block(x)
        
        # Output projection
        x = self.ln_f(x)
        logits = self.head(x)  # (B, T, vocab_size)
        
        # Compute loss if targets provided
        loss = None
        if targets is not None:
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1))
        
        return logits, loss
    
    @torch.no_grad()
    def generate(self, idx, max_new_tokens, temperature=1.0, top_k=None):
        """
        Generate new tokens autoregressively.
        
        Args:
            idx: (B, T) starting token indices
            max_new_tokens: number of tokens to generate
            temperature: sampling temperature (higher = more random)
            top_k: if set, only sample from top k most likely tokens
        """
        for _ in range(max_new_tokens):
            # Crop to block_size if needed
            idx_cond = idx[:, -self.block_size:]
            
            # Get predictions
            logits, _ = self(idx_cond)
            logits = logits[:, -1, :] / temperature  # (B, vocab_size)
            
            # Optional top-k filtering
            if top_k is not None:
                v, _ = torch.topk(logits, top_k)
                logits[logits < v[:, [-1]]] = float('-inf')
            
            # Sample from distribution
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            
            # Append to sequence
            idx = torch.cat([idx, idx_next], dim=1)
        
        return idx

In [ ]:
# Create model
model_config = {
    'vocab_size': vocab_size,
    'd_model': 128,
    'n_heads': 4,
    'n_layers': 4,
    'block_size': block_size,
    'dropout': 0.1,
}

model = GPT(**model_config).to(device)

# Count parameters
n_params = sum(p.numel() for p in model.parameters())
print(f"Model parameters: {n_params:,}")
print(f"\nConfig: {model_config}")

## 4. Training Loop

In [ ]:
def compute_loss(model, loader, device, max_batches=None):
    """Compute average loss on a dataset."""
    model.train(False)  # Set to inference mode
    total_loss = 0
    n_batches = 0
    
    with torch.no_grad():
        for i, (x, y) in enumerate(loader):
            if max_batches and i >= max_batches:
                break
            x, y = x.to(device), y.to(device)
            _, loss = model(x, y)
            total_loss += loss.item()
            n_batches += 1
    
    return total_loss / n_batches

def generate_sample(model, start_text, max_tokens=100, temperature=0.8):
    """Generate a text sample."""
    model.train(False)  # Set to inference mode
    
    idx = torch.tensor([encode(start_text)], device=device)
    generated = model.generate(idx, max_new_tokens=max_tokens, temperature=temperature)
    return decode(generated[0].tolist())

In [ ]:
# Training setup
learning_rate = 3e-4
n_epochs = 10
log_interval = 100
sample_interval = 500

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# Training history
train_losses = []
val_losses = []
steps = []

# Initial loss
print(f"Initial val loss: {compute_loss(model, val_loader, device, max_batches=50):.4f}")
print(f"Random baseline: {np.log(vocab_size):.4f} (= ln({vocab_size}))")
print()

In [ ]:
# Training loop
step = 0

for epoch in range(n_epochs):
    model.train()
    
    for batch_idx, (x, y) in enumerate(train_loader):
        x, y = x.to(device), y.to(device)
        
        # Forward pass
        logits, loss = model(x, y)
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        
        step += 1
        
        # Log progress
        if step % log_interval == 0:
            val_loss = compute_loss(model, val_loader, device, max_batches=20)
            train_losses.append(loss.item())
            val_losses.append(val_loss)
            steps.append(step)
            
            perplexity = np.exp(val_loss)
            print(f"Epoch {epoch+1} | Step {step:5d} | Train Loss: {loss.item():.4f} | "
                  f"Val Loss: {val_loss:.4f} | Perplexity: {perplexity:.2f}")
            
            model.train()  # Back to training mode
        
        # Generate sample
        if step % sample_interval == 0:
            print("\n" + "="*50)
            print("Generated sample:")
            sample = generate_sample(model, "\n", max_tokens=150, temperature=0.8)
            print(sample)
            print("="*50 + "\n")
            model.train()

In [ ]:
# Plot training curves
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Loss curve
axes[0].plot(steps, train_losses, label='Train', alpha=0.7)
axes[0].plot(steps, val_losses, label='Validation')
axes[0].axhline(y=np.log(vocab_size), color='gray', linestyle='--', label='Random baseline')
axes[0].set_xlabel('Step')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training Loss')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Perplexity curve
perplexities = [np.exp(l) for l in val_losses]
axes[1].plot(steps, perplexities)
axes[1].axhline(y=vocab_size, color='gray', linestyle='--', label='Random baseline')
axes[1].set_xlabel('Step')
axes[1].set_ylabel('Perplexity')
axes[1].set_title('Validation Perplexity')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nFinal validation loss: {val_losses[-1]:.4f}")
print(f"Final perplexity: {perplexities[-1]:.2f}")

## 5. Text Generation

In [ ]:
# Generate with different temperatures
print("="*60)
print("GENERATION WITH DIFFERENT TEMPERATURES")
print("="*60)

prompt = "\nTo be, or not to be"

for temp in [0.5, 0.8, 1.0, 1.5]:
    print(f"\n--- Temperature {temp} ---")
    sample = generate_sample(model, prompt, max_tokens=100, temperature=temp)
    print(sample)

print("\n" + "="*60)
print("Lower temperature = more deterministic, repetitive")
print("Higher temperature = more random, creative")
print("="*60)

In [ ]:
# Generate longer samples
print("="*60)
print("LONGER GENERATION (300 characters)")
print("="*60)

prompts = [
    "\nROMEO:\n",
    "\nThe king doth keep",
    "\nWhat light through yonder"
]

for prompt in prompts:
    print(f"\n--- Prompt: {repr(prompt)} ---")
    sample = generate_sample(model, prompt, max_tokens=300, temperature=0.8)
    print(sample)
    print()

## 6. Understanding Perplexity

Perplexity measures how "surprised" the model is by the data:

$$\text{Perplexity} = e^{\text{loss}} = e^{-\frac{1}{N}\sum_{i=1}^N \log p(x_i)}$$

- **Lower is better**
- Perplexity of $k$ means the model is as uncertain as if choosing uniformly among $k$ options
- Random baseline: perplexity = vocab_size

In [ ]:
# Compute per-character probabilities
model.train(False)  # Set to inference mode

test_text = "\nTo be, or not to be, that is the question:\n"
test_tokens = torch.tensor([encode(test_text)], device=device)

with torch.no_grad():
    logits, _ = model(test_tokens)
    probs = F.softmax(logits, dim=-1)

# Get probability of each actual next character
print("Character-by-character probabilities:")
print("-" * 60)

for i in range(min(30, len(test_text) - 1)):
    char = test_text[i]
    next_char = test_text[i + 1]
    next_idx = encode(next_char)[0]
    prob = probs[0, i, next_idx].item()
    
    # Get top prediction
    top_idx = probs[0, i].argmax().item()
    top_char = idx_to_char[top_idx]
    
    bar = "#" * int(prob * 40)
    print(f"'{char}' -> '{next_char}' (p={prob:.3f}) {bar}")
    if top_char != next_char:
        print(f"      (top prediction: '{top_char}')")

## 7. Visualizing Attention Patterns

In [ ]:
# Modified attention layer to capture weights
def get_attention_weights(model, text):
    """Get attention weights for a piece of text."""
    model.train(False)  # Set to inference mode
    
    tokens = torch.tensor([encode(text)], device=device)
    
    # Hook to capture attention weights
    attention_weights = []
    
    def hook(module, input, output):
        # Get attention scores before softmax
        B, T, C = input[0].shape
        qkv = module.qkv(input[0])
        q, k, v = qkv.split(module.d_model, dim=2)
        
        q = q.view(B, T, module.n_heads, module.d_k).transpose(1, 2)
        k = k.view(B, T, module.n_heads, module.d_k).transpose(1, 2)
        
        attn = (q @ k.transpose(-2, -1)) * module.scale
        attn = attn.masked_fill(module.mask[:, :, :T, :T] == 0, float('-inf'))
        attn = F.softmax(attn, dim=-1)
        
        attention_weights.append(attn.detach().cpu())
    
    # Register hooks
    hooks = []
    for block in model.blocks:
        h = block.attn.register_forward_hook(hook)
        hooks.append(h)
    
    # Forward pass
    with torch.no_grad():
        _ = model(tokens)
    
    # Remove hooks
    for h in hooks:
        h.remove()
    
    return attention_weights, text

# Get attention weights
test_text = "To be or not"
attn_weights, text = get_attention_weights(model, test_text)

print(f"Text: '{text}'")
print(f"Number of layers: {len(attn_weights)}")
print(f"Attention shape per layer: {attn_weights[0].shape}")

In [ ]:
# Visualize attention patterns
def plot_attention(attn_weights, text, layer=0, head=0):
    """Plot attention pattern for a specific layer and head."""
    attn = attn_weights[layer][0, head].numpy()  # (T, T)
    chars = list(text)
    
    fig, ax = plt.subplots(figsize=(8, 6))
    im = ax.imshow(attn, cmap='Blues')
    
    ax.set_xticks(range(len(chars)))
    ax.set_yticks(range(len(chars)))
    ax.set_xticklabels(chars)
    ax.set_yticklabels(chars)
    
    ax.set_xlabel('Key (attending to)')
    ax.set_ylabel('Query (from)')
    ax.set_title(f'Attention - Layer {layer}, Head {head}')
    
    plt.colorbar(im, ax=ax)
    plt.tight_layout()
    return fig

# Plot different layers and heads
fig = plot_attention(attn_weights, test_text, layer=0, head=0)
plt.show()

fig = plot_attention(attn_weights, test_text, layer=3, head=0)
plt.show()

print("Notice: The causal mask makes the upper triangle all zeros.")
print("Each position can only attend to itself and earlier positions.")

In [ ]:
# Compare all heads in a layer
layer = 0
n_heads = model_config['n_heads']

fig, axes = plt.subplots(1, n_heads, figsize=(4 * n_heads, 4))

for head in range(n_heads):
    attn = attn_weights[layer][0, head].numpy()
    im = axes[head].imshow(attn, cmap='Blues')
    axes[head].set_title(f'Head {head}')
    axes[head].set_xticks(range(len(test_text)))
    axes[head].set_yticks(range(len(test_text)))
    axes[head].set_xticklabels(list(test_text), fontsize=8)
    axes[head].set_yticklabels(list(test_text), fontsize=8)

plt.suptitle(f'All Attention Heads - Layer {layer}')
plt.tight_layout()
plt.show()

## 8. Model Analysis

In [ ]:
# Parameter breakdown
def count_params(model):
    """Count parameters by component."""
    counts = {
        'embeddings': model.tok_emb.weight.numel() + model.pos_emb.weight.numel(),
        'attention': sum(sum(p.numel() for p in block.attn.parameters()) for block in model.blocks),
        'feedforward': sum(sum(p.numel() for p in block.ffn.parameters()) for block in model.blocks),
        'layer_norms': sum(
            sum(p.numel() for p in block.ln1.parameters()) +
            sum(p.numel() for p in block.ln2.parameters())
            for block in model.blocks
        ) + sum(p.numel() for p in model.ln_f.parameters()),
    }
    # Note: output head shares weights with embedding
    return counts

counts = count_params(model)
total = sum(counts.values())

print("Parameter Breakdown:")
print("-" * 40)
for name, count in counts.items():
    pct = count / total * 100
    print(f"{name:20s} {count:10,d} ({pct:5.1f}%)")
print("-" * 40)
print(f"{'Total':20s} {total:10,d}")

In [ ]:
# Compare to theoretical FLOPS
d = model_config['d_model']
T = model_config['block_size']
L = model_config['n_layers']
V = model_config['vocab_size']

print("Approximate FLOPS per forward pass:")
print(f"  Embedding lookup: O(T * d) = {T * d:,}")
print(f"  Attention (per layer): O(T^2 * d) = {T * T * d:,}")
print(f"  FFN (per layer): O(T * d^2) = {T * d * d * 4:,}")
print(f"  Output projection: O(T * V * d) = {T * V * d:,}")
print()
print(f"Total per layer: ~{T * T * d + T * d * d * 4:,}")
print(f"Total for {L} layers: ~{L * (T * T * d + T * d * d * 4):,}")

## Summary

| Component | Purpose |
|-----------|--------|
| Token Embedding | Convert tokens to vectors |
| Position Embedding | Inject position information |
| Causal Attention | Attend only to past tokens |
| Weight Tying | Share embedding/output weights |

**Key Insights:**
1. GPT is just a transformer decoder with **causal masking**
2. **Weight tying** between embedding and output reduces parameters
3. **Temperature** controls generation randomness
4. **Perplexity** measures model uncertainty (lower is better)
5. Even tiny models can learn character patterns from Shakespeare

**Scaling:** Real GPT models use:
- Billions of parameters (we have ~500K)
- Much longer context (8K-128K vs our 64)
- Trained on trillions of tokens (we have ~1M)

**Next:** [10-fine-tuning-basics.ipynb](10-fine-tuning-basics.ipynb) explores fine-tuning pretrained models.

## Exercises

1. **Larger Model:** Increase d_model to 256 and n_layers to 6. How does this affect training speed and quality?

2. **Top-k Sampling:** Implement top-k sampling in the generate method. Compare outputs with k=5, 10, 50.

3. **Learning Rate Schedule:** Implement cosine learning rate decay. Does it improve final perplexity?

4. **Different Corpus:** Train on a different text (song lyrics, code, poetry). How does the output style change?